In [3]:
from urllib import request
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer, LancasterStemmer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/amarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/amarov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/amarov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

First we divide the articles into individual sentences by splitting the string by ".".

## Text normalisation

### Sentence tokenization


In [4]:
ballet_url = "https://raw.githubusercontent.com/boyko/text-analytics-script/main/data/ballet.txt"
url_handle = request.urlopen(ballet_url)
articles = url_handle.read().decode("utf-8")

In [5]:
naive_sentences = re.split(r"[.!?] ", articles)
sentences = nltk.sent_tokenize(articles)
print(sentences[:10])

['With apologies to James Brown, the hardest working people in show business may well be ballet dancers.', 'And at New York City Ballet, none work harder than the dancers in its lowest rank, the corps de ballet.', 'During the first week of the company’s winter season, Claire Kretzschmar, 24, a rising corps member, danced in all seven performances, appearing in five ballets, sometimes changing costumes at intermission to dance two roles in a night.', 'But her work onstage did not even begin to capture the stamina required to be in the corps.', 'Spending a week shadowing Ms. Kretzschmar was exhausting — she gave new meaning to the idea of being on your feet all day.', 'Twelve-hour days at the David H. Koch Theater, the company’s Lincoln Center home, were hardly unusual: Company class each morning was followed by back-to-back-to-back rehearsals, with occasional breaks for costume fittings or physical therapy, and then by the hair-makeup-costume-dance routine of daily performances.', 'Vide

In [16]:
## Compare with naive split on "."
print(
    f"Number of sentences from sent_tokenize: {len(sentences)}. Number of sentences from naive split: {len(naive_sentences)}."
)

Number of sentences from sent_tokenize: 91. Number of sentences from naive split: 116.



### Word tokenization

Splitting a sentence into words.

In [6]:
words = [nltk.word_tokenize(w) for w in sentences]


An alternative is to use [spacy](https://spacy.io/usage/linguistic-features#how-tokenizer-works).

### Stemming and Lemmatization

The basic independent unit in natural languages are the _morphemes_. These arise by inflecting
a stem with some affix (e.g. prefixes, suffixes)

- *Stemming*: find the stem of the word
- *Lemmatization*: find the base form (lemma) of the word by removing affixes. The root form of the word must be present in the dictionary.

In [7]:
p_stemmer = PorterStemmer()
lc_stemmer = LancasterStemmer()

p_stemmer.stem("strange")

'strang'

In [8]:
lc_stemmer.stem("strange")

'strange'

To reduce all words to their base form we can use a lemmatizer. The `nltk` package provides


In order for the lemmatizer to work it requires the POS tags of the words in the sentence.
The `WorldNetLemmatizer`

In [9]:
wn_lemmatizer = nltk.WordNetLemmatizer()
wn_lemmatizer.lemmatize('cars', 'n')

'car'

In [11]:
wn_lemmatizer = nltk.WordNetLemmatizer()
wn_lemmatizer.lemmatize('running', 'v')

'running'

The word is returned unchanged if no match is found. The default pos tag is "n" (noun).

In [90]:
wn_lemmatizer.lemmatize("gsdgfsdf")

'gsdgfsdf'

## Part of speech tagging

Part of speech (POS) tagging refers to determining the role of each word within a sentence, e.g.:

1. CD  Cardinal number
    e.g. 1, 20
2. DT  Determiner
    e.g. a/an, the, 2, some, many
3. JJ  Adjective
    e.g. good, big, red
4. JJR Adjective, comparative
5. MD  Modal
    e.g. can, must, may, might, will, would, should
6. NN  Noun, singular or mass
    e.g. day, cat
7. NNS Noun, plural
    e.g. cats, dogs
8. RB  Adverb
    e.g. quickly, silently, well, badly, very, really
9. RBR Adverb, comparative
    e.g. cheaper/more cheaply, slower/more slowly
10. RBS Adverb, superlative
    e.g. fastest, hardest
11. VB  Verb, base form
    e.g. be, have
12. WP  Wh-pronoun
    e.g. I, you, he, she, some
13. WP$ Possessive wh-pronoun
    e.g. mine, his, her, your, yours

for the full list of Penn Treebank POS tags see their [web site](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html).

Here we use the default POS tagger from `nltk` to label each word.

In [13]:
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

from nltk import word_tokenize, pos_tag
from collections import defaultdict
tag_map = defaultdict(lambda: wn.NOUN, J = wn.ADV, V = wn.VERB, R = wn.ADV)

sentence = "It is better to be hated for what you are than to be loved for what you are not."

tokens = word_tokenize(sentence)
lemmatizer = WordNetLemmatizer()
for token, tag in pos_tag(tokens):
    lemma = lemmatizer.lemmatize(token, tag_map[tag[0]])
    print(token, "(", tag, ")", "-->", lemma)

It ( PRP ) --> It
is ( VBZ ) --> be
better ( RBR ) --> well
to ( TO ) --> to
be ( VB ) --> be
hated ( VBN ) --> hat
for ( IN ) --> for
what ( WP ) --> what
you ( PRP ) --> you
are ( VBP ) --> be
than ( IN ) --> than
to ( TO ) --> to
be ( VB ) --> be
loved ( VBN ) --> love
for ( IN ) --> for
what ( WP ) --> what
you ( PRP ) --> you
are ( VBP ) --> be
not ( RB ) --> not
. ( . ) --> .


In [99]:
tag_map

defaultdict(<function __main__.<lambda>()>,
            {'J': 'r',
             'V': 'v',
             'R': 'r',
             'P': 'n',
             'T': 'n',
             'I': 'n',
             'W': 'n',
             '.': 'n'})

In [85]:
import spacy

nlp = spacy.load("en_core_web_md")
lemmatizer = nlp.get_pipe("lemmatizer")
doc = nlp(sentence)

print([token.lemma_ for token in doc])

['it', 'be', 'well', 'to', 'be', 'hate', 'for', 'what', 'you', 'be', 'than', 'to', 'be', 'love', 'for', 'what', 'you', 'be', 'not', '.']


### Stop words

Natural language contains a lot of words that appear very frequently in all text but that contribute little
to the meaning of the text, for example: "and", "the", etc. Such words are commonly removed from the text in the process of normalization.


In [36]:
stopwords_en = nltk.corpus.stopwords.words("english")
stopwords_en

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

One way to remove the stop words from a list of words is to simply use list comprehension.

In [35]:
words = "A room without books is like a body without a soul. Cicero.".lower()
list_of_words = words.split(" ")

list_of_words_filtered = list([w for w in list_of_words if w not in stopwords_en])
list_of_words_filtered

['room', 'without', 'books', 'like', 'body', 'without', 'soul.', 'cicero.']